In [4]:
import sys
import os

from pathlib import Path

data_path = Path("../data")
os.environ["VECTORIAN_DEMO_DATA_PATH"] = str(data_path)

sys.path.append("code")
import nbutils
import json

from vectorian.importers import TextImporter
from vectorian.embeddings import SpacyVectorEmbedding
from vectorian.corpus import Corpus
from tqdm import tqdm

nlp = nbutils.make_nlp("en_paraphrase_distilroberta_base_v1")

with open(data_path / "raw_data" / "gold.json", "r") as f:
    queries = json.loads(f.read())

bert_embedding = SpacyVectorEmbedding(nlp, 768)

def prepare_docs():
    if bert_embedding is not None:
        im = TextImporter(nlp, embeddings=[bert_embedding])
    else:
        im = TextImporter(nlp)

    corpus = Corpus(data_path / "processed_data"/ "corpus")

    for query in tqdm(queries, desc="Importing"):
        for m in query["matches"]:
            doc = im(
                m["context"],
                title=m["work"],
                author=m["author"],
                extra_metadata={
                    'gold_id': m["id"]
                },
                show_progress=False)

            corpus.add_doc(doc)

prepare_docs()

Importing: 100%|██████████| 20/20 [03:15<00:00,  9.77s/it]
